In [17]:
def load_hdf_group(data_dir, hdf_filename, group="/"):
    '''
    Loads any datasets from the given hdf group into a dictionary. Also will
    recursively load other groups if any exist under the given group

    Args:
        data_dir (str): folder where data is located
        hdf_filename (str): name of hdf file
        group (str): name of the group to load
    
    Returns:
        dict: all the datasets contained in the given group
    '''
    full_file_name = os.path.join(data_dir, hdf_filename)
    hdf = h5py.File(full_file_name, 'r')
    if group not in hdf:
        raise ValueError('No such group in file {}'.format(hdf_filename))

    # Recursively load groups until datasets are reached
    def _load_hdf_group(hdf):
        keys = hdf.keys()
        data = dict()
        for k in keys:
            if isinstance(hdf[k], h5py.Group):
                data[k] = _load_hdf_group(hdf[k])
            else:
                k_, v = _load_hdf_dataset(hdf[k], k)
                data[k_] = v
        return data

    data = _load_hdf_group(hdf[group])
    hdf.close()
    return data

def _load_hdf_dataset(dataset, name):
    '''
    Internal function for loading hdf datasets. Decodes json and unicode data automatically.

    Args:
        dataset (hdf object): dataset to load
        name (str): name of the dataset

    Returns:
        tuple: Tuple containing:
            | **name (str):** name of the dataset (might be modified)
            | **data (object):** loaded data
    '''
    data = dataset[()]
    if '_json' in name:
        import json
        name = name.replace('_json', '')
        data = json.loads(data)
    try:
        data = data.decode('utf-8')
    except:
        pass
    return name, data

In [18]:
import numpy as np
import matplotlib.pyplot as plt
import aopy
import os
import pandas as pds
from db import dbfunctions as db
from ipywidgets import interactive, widgets
import scipy
import h5py
from tqdm.auto import tqdm 
import seaborn as sn
import sklearn
from sklearn.decomposition import PCA, FactorAnalysis
from itertools import compress
import multiprocessing as mp
import time
import math
from scipy.fft import fft
import glob
from datetime import date

# Set parameters

In [19]:
save_figs = False
base_save_dir = "/media/moor-data/results/Ryan/neuropixel_targeting/"
np_preproc_data_folder = 'np_analysis_preproc_data'
ecog_dec_acc_file_name = 'ecog_decoding_maps/npinsert_ecog_decoding'

subject = 'beignet'
align_events = ['TARGET ONSET', 'GO CUE', 'MOVEMENT ONSET']

In [20]:
# Decoding calculation parameters
tbefore = 0.5
tafter = 1
nlda_lags = 1
niter_match = 50
min_trial_prop = .85
ntrial_bin_size = 96
nfolds = 4

# Visualization parameters
colors = sn.color_palette(n_colors=9)
recording_brain_areas={'M1': [30, 56, 47, 40, 121, 48, 120, 98], 'PM':[11, 9, 18, 22, 10, 45]}
day_colors = ['dodgerblue', 'indigo', 'violet', 'lightblue', 'mediumorchid',
              'purple', 'steelblue', 'dodgerblue', 'lightblue', 'red', 'black', 'green', 'purple', 'cyan', 'gray', 'yellow'] 

save_dir = "/media/moor-data/results/Ryan/neuropixel_targeting/np_analysis_preproc_data"

# Load and extract relevant data

In [21]:
ecog_dec_acc = load_hdf_group(base_save_dir, ecog_dec_acc_file_name)

In [22]:
start = time.time()
df, rasters, preproc_metadata = aopy.data.base.pkl_read(f"{subject}_np_preprocessed", os.path.join(base_save_dir, np_preproc_data_folder))
print(f"{np.round((time.time()-start)/60)} min to load preprocessed data")
nrecs = preproc_metadata['nrecs']
recording_site = preproc_metadata['recording_sites'] # will be the same for all align events
implants = ['NPinsert72' if preproc_metadata['implant'][irec] == 'NP_Insert72' else 'NPinsert137' for irec in range(len(preproc_metadata['implant']))] #Rename because name in bmi3d is slightly different (TODO)
dates = np.unique(df['date'])

3.0 min to load preprocessed data


In [23]:
qc_results, ksdrift = aopy.data.base.pkl_read(f"{subject}_QCunits", os.path.join(base_save_dir, np_preproc_data_folder))
stable_unit_labels = [qc_results['final_good_unit_labels'][irec] for irec in range(nrecs)]
stable_unit_idx = [qc_results['final_good_unit_idx'][irec] for irec in range(nrecs)]
nstable_unit = np.array([len(qc_results['final_good_unit_idx'][irec]) for irec in range(nrecs)])
neuron_pos = [qc_results['position'][irec] for irec in range(nrecs)]

In [24]:
# compile dataframe of unit information
# Need columns for: date, recording site, unit_label, absolute y-pos, relative y-pos, waveform
unit_info = {'rec_number': [], 'date': [], 'rec_site': [], 'rec_xpos': [], 'rec_ypos': [], 'implant': [],
             'unit_label': [], 'unit_idx': [], 'abs_depth': [], 'rel_depth': [], 'waveform': []}
for irec in tqdm(range(nrecs)):
    for iunit in range(nstable_unit[irec]):
        unit_info['rec_number'].append(irec) # Recording_number
        unit_info['date'].append(np.unique(df['date'])[irec]) # date (assumes each recording is done on a different day)
        unit_info['rec_site'].append(recording_site[irec]) # Recording site
        unit_info['rec_xpos'].append(ecog_dec_acc[subject]['rec_locations'][irec,0]) # Recording site x pos in chamber
        unit_info['rec_ypos'].append(ecog_dec_acc[subject]['rec_locations'][irec,1]) # Recording site y pos in chamber
        unit_info['implant'].append(implants[irec]) # Implant
        unit_info['unit_label'].append(stable_unit_labels[irec][iunit]) # unit label output from kilosort
        unit_info['unit_idx'].append(stable_unit_idx[irec][iunit]) # unit label output from kilosort
        unit_info['abs_depth'].append(3840 - neuron_pos[irec][iunit]) #Absolute depth [um]
        unit_info['rel_depth'].append(3840-neuron_pos[irec][iunit]-np.min(3840-neuron_pos[irec])) #Relative depth [um] (to top detected neuron)
        unit_info['waveform'].append(qc_results['mean_wfs'][irec][iunit,:]) #waveform

  0%|          | 0/15 [00:00<?, ?it/s]

In [25]:
unit_df = pds.DataFrame(unit_info)

# Create pseudopopulations

In [26]:
pp_configs = ['random', 'column', 'depth'] # Pseudopopulation cofigurations 

## Random

In [33]:
# Create random neuron populations
nrandom_units = 13 # Number of random units chosen in each group
nrandom_groups = 500 # Number of random groups to make
random_group_info = []
for igroup in range(nrandom_groups):
    group_idx = np.random.choice(np.arange(len(unit_df)), size=(nrandom_units), replace=False)
    random_group_info.append(unit_df.loc[group_idx])

## Column

In [34]:
# Create random neuron populations from each recording_location
unique_rec_pos = np.unique(ecog_dec_acc[subject]['rec_locations'], axis=0)
column_group_info = []
for ipos in range(unique_rec_pos.shape[0]):
    pos = unique_rec_pos[ipos,:]
    column_group_info.append(unit_df.loc[(unit_df['rec_xpos']==pos[0]) & (unit_df['rec_ypos']==pos[1])].reset_index())

## Depth

In [35]:
# Create random neuron populations from shallow and deep neurons
depth_ranges = [(0,1000), (1000, 2000), (2000, 100000)] #Boundary distinguishing shallow and deep neurons 
depth_group_info = []
for irange, depth_range in enumerate(depth_ranges):
    depth_group_info.append(unit_df.loc[(unit_df['rel_depth'] >= depth_range[0]) & (unit_df['rel_depth'] < depth_range[1])].reset_index())

# Save

In [36]:
pseudopopulation_metadata = {'nrandom_units': nrandom_units, 'nrandom_groups': nrandom_groups, 'depth_ranges': depth_ranges}

In [42]:
# Save data
aopy.data.base.pkl_write(f"{subject}_np_psuedopopulations", (random_group_info, column_group_info, depth_group_info, pseudopopulation_metadata, unit_df), save_dir)

In [38]:
print(len(depth_group_info[0]))
print(len(depth_group_info[1]))
print(len(depth_group_info[2]))


154
145
325
